## Домашняя работа по теме "Part-of-Speech разметка, NER, извлечение отношений"

In [2]:
import pandas as pd
from corus import load_ne5
from razdel import tokenize

from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


2023-08-26 23:00:24.039352: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 23:00:24.272845: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 23:00:24.281635: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 23:00:29.294624: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Загрузка данных

In [3]:
dir = 'collection5/Collection5/'
records = load_ne5(dir)
next(records)

Ne5Markup(
    id='1040',
    text='Игнатьев: покупка валюты Минфином не повлияет на динамику курса рубля\r\n\r\n\r\nПокупка Минфином РФ валюты для пополнения госфондов не окажет влияния на динамику курса рубля, заявил глава ЦБ Сергей Игнатьев. Министерство уже в августе начнет покупку иностранной валюты.\r\n\r\n\r\n Покупка Минфином РФ валюты для пополнения госфондов не окажет влияния на динамику курса рубля, заявил глава ЦБ Сергей Игнатьев, выступая в Госдуме.\r\n\r\n\r\n\r\nГлава Минфина Антон Силуанов в понедельник заявил, что министерство уже в августе начнет покупку иностранной валюты на внутреннем рынке для пополнения госфондов, что приведет к ослаблению курса на 1-2 рубля. На этом фоне курс рубля к доллару и евро после нескольких дней роста вновь начал падать.\r\n\r\n"Действительно, мы подготовили новый порядок валютных интервенций, по которому Минфин будет покупать в ежедневном режиме валюту, а ЦБ будет в тех же объемах покупать на рынке <…> Этот порядок будет запущен в август

Преобразование к датафрейму

In [4]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [5]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [6]:
df_words['tag'].value_counts()

OUT         218709
PER          21185
ORG          13628
LOC           4568
GEOPOLIT      4351
MEDIA         2482
Name: tag, dtype: int64

In [7]:
df_words_list = list(df_words.itertuples(index=False, name=None))

Преобразование к листам с предложениям

In [8]:
sentences = []
one_sentence = []
for i in range(len(df_words_list)):
    if df_words_list[i] != ('.', 'OUT'):
        one_sentence.append(df_words_list[i])
    if df_words_list[i] == ('.', 'OUT'):
        one_sentence.append(df_words_list[i])
        sentences.append(one_sentence)
        one_sentence = []

In [10]:
sentences[:3]

[[('Новым', 'OUT'),
  ('президентом', 'OUT'),
  ('Германии', 'GEOPOLIT'),
  ('стал', 'OUT'),
  ('Иоахим', 'PER'),
  ('Гаук', 'PER'),
  ('Бывший', 'OUT'),
  ('лютеранский', 'OUT'),
  ('священник', 'OUT'),
  ('и', 'OUT'),
  ('правозащитник', 'OUT'),
  ('Иоахим', 'PER'),
  ('Гаук', 'PER'),
  ('избран', 'OUT'),
  ('на', 'OUT'),
  ('должность', 'OUT'),
  ('президента', 'OUT'),
  ('ФРГ', 'GEOPOLIT'),
  ('.', 'OUT')],
 [('Об', 'OUT'),
  ('этом', 'OUT'),
  ('передает', 'OUT'),
  ('Reuters', 'MEDIA'),
  ('со', 'OUT'),
  ('ссылкой', 'OUT'),
  ('на', 'OUT'),
  ('спикера', 'OUT'),
  ('парламента', 'OUT'),
  ('республики', 'OUT'),
  ('Норберта', 'PER'),
  ('Ламмерта', 'PER'),
  ('.', 'OUT')],
 [('Накануне', 'OUT'),
  ('голосования', 'OUT'),
  ('высказывались', 'OUT'),
  ('предположения', 'OUT'),
  (',', 'OUT'),
  ('что', 'OUT'),
  ('И', 'PER'),
  ('.', 'PER'),
  ('Гаук', 'PER'),
  ('имеет', 'OUT'),
  ('все', 'OUT'),
  ('шансы', 'OUT'),
  ('на', 'OUT'),
  ('то', 'OUT'),
  (',', 'OUT'),
  ('чтобы', '

In [11]:
train_data = sentences[:int(len(sentences) * 0.9)]
test_data = sentences[int(len(sentences) * 0.9):]
test_data[0]

[('В', 'OUT'),
 ('настоящее', 'OUT'),
 ('время', 'OUT'),
 ('проводится', 'OUT'),
 ('проверка', 'OUT'),
 ('на', 'OUT'),
 ('предмет', 'OUT'),
 ('наличия', 'OUT'),
 ('в', 'OUT'),
 ('действиях', 'OUT'),
 ('должностных', 'OUT'),
 ('лиц', 'OUT'),
 ('Роскосмоса', 'ORG'),
 ('состава', 'OUT'),
 ('преступления', 'OUT'),
 (',', 'OUT'),
 ('предусмотренного', 'OUT'),
 ('ст', 'OUT'),
 ('.', 'OUT')]

In [12]:
test_sent = [word for word, tag in test_data[0]]

In [14]:
results = []

#### UnigramTagger

UnigramTagger учитывает условную частоту тегов и предсказывает наиболее частый тег для каждого токена, не ориентируется на соседние слова.

In [15]:
unigram_tagger = UnigramTagger(train_data)
unigram_tagger_result = ('unigram_tagger', unigram_tagger.evaluate(test_data))
results.append(unigram_tagger_result)
display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))

/tmp/ipykernel_7927/1424434567.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  unigram_tagger_result = ('unigram_tagger', unigram_tagger.evaluate(test_data))
/tmp/ipykernel_7927/1424434567.py:4: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(test_data))


[('В', 'OUT'),
 ('настоящее', 'OUT'),
 ('время', 'OUT'),
 ('проводится', 'OUT'),
 ('проверка', 'OUT'),
 ('на', 'OUT'),
 ('предмет', 'OUT'),
 ('наличия', 'OUT'),
 ('в', 'OUT'),
 ('действиях', 'OUT'),
 ('должностных', 'OUT'),
 ('лиц', 'OUT'),
 ('Роскосмоса', 'ORG'),
 ('состава', 'OUT'),
 ('преступления', 'OUT'),
 (',', 'OUT'),
 ('предусмотренного', 'OUT'),
 ('ст', 'OUT'),
 ('.', 'OUT')]

0.8747254411876089

#### BigramTagger

BigramTagger будет учитывает тэги двух слов: текущее и предыдущее слово. Точность немного выше, чем у Unigram tagger.

In [18]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)
bigram_tagger_result = ('bigram_tagger', bigram_tagger.evaluate(test_data))
results.append(bigram_tagger_result)                             
display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(test_data))

/tmp/ipykernel_7927/808695760.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  bigram_tagger_result = ('bigram_tagger', bigram_tagger.evaluate(test_data))
/tmp/ipykernel_7927/808695760.py:4: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(test_data))


[('В', 'OUT'),
 ('настоящее', 'OUT'),
 ('время', 'OUT'),
 ('проводится', 'OUT'),
 ('проверка', 'OUT'),
 ('на', 'OUT'),
 ('предмет', 'OUT'),
 ('наличия', 'OUT'),
 ('в', 'OUT'),
 ('действиях', 'OUT'),
 ('должностных', 'OUT'),
 ('лиц', 'OUT'),
 ('Роскосмоса', 'ORG'),
 ('состава', 'OUT'),
 ('преступления', 'OUT'),
 (',', 'OUT'),
 ('предусмотренного', 'OUT'),
 ('ст', 'OUT'),
 ('.', 'OUT')]

0.8817314246762099

#### TrigramTagger

In [20]:
trigram_tagger = TrigramTagger(train_data, backoff=bigram_tagger)
trigram_tagger_result = ('trigram_tagger', trigram_tagger.evaluate(test_data))
results.append(trigram_tagger_result)                  
display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(test_data))

/tmp/ipykernel_7927/1049589636.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  trigram_tagger_result = ('trigram_tagger', trigram_tagger.evaluate(test_data))
/tmp/ipykernel_7927/1049589636.py:4: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(test_data))


[('В', 'OUT'),
 ('настоящее', 'OUT'),
 ('время', 'OUT'),
 ('проводится', 'OUT'),
 ('проверка', 'OUT'),
 ('на', 'OUT'),
 ('предмет', 'OUT'),
 ('наличия', 'OUT'),
 ('в', 'OUT'),
 ('действиях', 'OUT'),
 ('должностных', 'OUT'),
 ('лиц', 'OUT'),
 ('Роскосмоса', 'ORG'),
 ('состава', 'OUT'),
 ('преступления', 'OUT'),
 (',', 'OUT'),
 ('предусмотренного', 'OUT'),
 ('ст', 'OUT'),
 ('.', 'OUT')]

0.8872983412860713

In [22]:
train_tok = []
train_label = []
for sent in train_data[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in test_data[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [23]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [24]:
test_enc_labels = le.transform(test_label)

#### Векторайзеры и логистическая регрессия

In [25]:
hvectorizer_char = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=50)

In [26]:
X_train = hvectorizer_char.fit_transform(train_tok)

In [27]:
X_test = hvectorizer_char.transform(test_tok)

In [28]:
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

/home/polina/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10, random_state=0)

In [29]:
pred = lr.predict(X_test)

In [30]:
accuracy_score(test_enc_labels, pred)

0.8290540028781338

In [31]:
hvectorizer_char_result = ('hvectorizer_char', accuracy_score(test_enc_labels, pred))
results.append(hvectorizer_char_result)       

In [32]:
hvectorizer_word = HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=50)

In [33]:
X_train = hvectorizer_word.fit_transform(train_tok)
X_test = hvectorizer_word.transform(test_tok)

In [34]:
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

/home/polina/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10, random_state=0)

In [35]:
pred = lr.predict(X_test)

In [36]:
accuracy_score(test_enc_labels, pred)

0.8275013254563357

In [37]:
hvectorizer_word_result = ('hvectorizer_word', accuracy_score(test_enc_labels, pred))
results.append(hvectorizer_word_result)      

#### Резюмируем результаты:

In [38]:
for i in range(len(results)):
    print(results[i])

('unigram_tagger', 0.8747254411876089)
('bigram_tagger', 0.8817314246762099)
('trigram_tagger', 0.8872983412860713)
('hvectorizer_char', 0.8290540028781338)
('hvectorizer_word', 0.8275013254563357)


__Вывод:__ лучше всего показатели у TrigramTagger